# Prediction of the number of passengers carried in air transport

## Data

The dataset provides an overview of the number of people who travelled with US airlines in a given month. 

It contains records for 142 months. Contains 2 columns "Month" and "Passengers".

## Task
Your task is to create different passenger prediction models
- Simple RNN
- LSTM
- GRU

For each model, compare the predicted and actual values.

Compare the models and choose the best one.

Experiment with the length of the learning sequence.

## Data loading and display

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import SimpleRNN, LSTM, GRU, Dense, Input
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
dataset = pd.read_csv("../dataset/airline-passengers.csv", index_col="Month", parse_dates=["Month"])

In [ ]:
dataset.head()

In [ ]:
dataset.describe()

In [ ]:
dataset.isna().sum()

Data split into 75% training and 25% testing

In [ ]:
train_size = int(len(dataset) * 0.75)
test_size = len(dataset) - train_size
print(f"Train size: {train_size}, Test size: {test_size}")

In [ ]:
train = dataset[:train_size]
test = dataset[train_size:]

Data display

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(train)
plt.plot(test)
plt.legend(["Train", "Test"])
plt.title("Number of passengers")
plt.show()

# Data preparation

Standardize data with MinMaxScaler

In [ ]:
sc = MinMaxScaler(feature_range=(0, 1))

training_data = train["Passengers"].values.reshape(-1, 1)
testing_data = test["Passengers"].values.reshape(-1, 1)

training_data_scaled = sc.fit_transform(training_data)
testing_data_scaled = sc.transform(testing_data)

From the data, make sequences

In [ ]:
def split_sequence(sequence, n_steps):
    X, y = [], []
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence) - 1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
n_steps = 10
features = 1

X_train, y_train = split_sequence(training_data_scaled, n_steps)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], features)

X_test, y_test = split_sequence(testing_data_scaled, n_steps)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], features)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

In [ ]:
def return_rmse(test, predicted):
    rmse = np.sqrt(mean_squared_error(test, predicted))
    return rmse

def plot_predictions(test, predicted, title):
    plt.figure(figsize=(10, 4))
    plt.plot(test, color="gray", label="Real")
    plt.plot(predicted, color="red", label="Predicted")
    plt.title(title)
    plt.xlabel("Time")
    plt.ylabel("Passengers")
    plt.legend()
    plt.show()

# Simple RNN

In [ ]:
model_rnn = Sequential()
model_rnn.add(Input(shape=(n_steps, features)))
model_rnn.add(SimpleRNN(units=50, activation="tanh"))
model_rnn.add(Dense(units=1))

model_rnn.compile(optimizer="adam", loss="mse", metrics=['mae'])
model_rnn.summary()

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='loss', patience=10)
history_rnn = model_rnn.fit(X_train, y_train, epochs=100, batch_size=8, callbacks=[early_stop], verbose=1)

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(history_rnn.history['loss'], label='Loss')
plt.plot(history_rnn.history['mae'], label='MAE')
plt.title('SimpleRNN - Learning History')
plt.legend()
plt.show()

## Validation of SimpleRNN model

In [ ]:
pred_rnn = model_rnn.predict(X_test)
pred_rnn = sc.inverse_transform(pred_rnn)
actual = testing_data[n_steps:]

rmse_rnn = return_rmse(actual, pred_rnn)
print(f"SimpleRNN RMSE: {rmse_rnn:.2f}")

In [ ]:
plot_predictions(actual, pred_rnn, "SimpleRNN - Passengers Prediction")

# LSTM neural network

In [ ]:
model_lstm = Sequential()
model_lstm.add(Input(shape=(n_steps, features)))
model_lstm.add(LSTM(units=50, activation="tanh"))
model_lstm.add(Dense(units=1))

model_lstm.compile(optimizer="adam", loss="mse", metrics=['mae'])
model_lstm.summary()

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='loss', patience=10)
history_lstm = model_lstm.fit(X_train, y_train, epochs=100, batch_size=8, callbacks=[early_stop], verbose=1)

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(history_lstm.history['loss'], label='Loss')
plt.plot(history_lstm.history['mae'], label='MAE')
plt.title('LSTM - Learning History')
plt.legend()
plt.show()

## LSTM model validation

In [ ]:
pred_lstm = model_lstm.predict(X_test)
pred_lstm = sc.inverse_transform(pred_lstm)

rmse_lstm = return_rmse(actual, pred_lstm)
print(f"LSTM RMSE: {rmse_lstm:.2f}")

In [ ]:
plot_predictions(actual, pred_lstm, "LSTM - Passengers Prediction")

# Model GRU

In [ ]:
model_gru = Sequential()
model_gru.add(Input(shape=(n_steps, features)))
model_gru.add(GRU(units=50, activation="tanh"))
model_gru.add(Dense(units=1))

model_gru.compile(optimizer="adam", loss="mse", metrics=['mae'])
model_gru.summary()

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='loss', patience=10)
history_gru = model_gru.fit(X_train, y_train, epochs=100, batch_size=8, callbacks=[early_stop], verbose=1)

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(history_gru.history['loss'], label='Loss')
plt.plot(history_gru.history['mae'], label='MAE')
plt.title('GRU - Learning History')
plt.legend()
plt.show()

## Validation of GRU model

In [ ]:
pred_gru = model_gru.predict(X_test)
pred_gru = sc.inverse_transform(pred_gru)

rmse_gru = return_rmse(actual, pred_gru)
print(f"GRU RMSE: {rmse_gru:.2f}")

In [ ]:
plot_predictions(actual, pred_gru, "GRU - Passengers Prediction")

# Model Comparison

In [ ]:
print("=" * 40)
print("MODEL COMPARISON")
print("=" * 40)
print(f"SimpleRNN RMSE: {rmse_rnn:.2f}")
print(f"LSTM RMSE:      {rmse_lstm:.2f}")
print(f"GRU RMSE:       {rmse_gru:.2f}")
print("=" * 40)

best_model = min([("SimpleRNN", rmse_rnn), ("LSTM", rmse_lstm), ("GRU", rmse_gru)], key=lambda x: x[1])
print(f"Best model: {best_model[0]} with RMSE {best_model[1]:.2f}")

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(actual, color="black", label="Real", linewidth=2)
plt.plot(pred_rnn, label=f"SimpleRNN (RMSE: {rmse_rnn:.2f})")
plt.plot(pred_lstm, label=f"LSTM (RMSE: {rmse_lstm:.2f})")
plt.plot(pred_gru, label=f"GRU (RMSE: {rmse_gru:.2f})")
plt.title("All Models Comparison")
plt.xlabel("Time")
plt.ylabel("Passengers")
plt.legend()
plt.show()

# Experiment with sequence length

In [ ]:
def train_and_evaluate(n_steps, model_type="LSTM"):
    X_tr, y_tr = split_sequence(training_data_scaled, n_steps)
    X_tr = X_tr.reshape(X_tr.shape[0], X_tr.shape[1], 1)
    
    X_te, y_te = split_sequence(testing_data_scaled, n_steps)
    X_te = X_te.reshape(X_te.shape[0], X_te.shape[1], 1)
    
    model = Sequential()
    model.add(Input(shape=(n_steps, 1)))
    
    if model_type == "SimpleRNN":
        model.add(SimpleRNN(units=50, activation="tanh"))
    elif model_type == "LSTM":
        model.add(LSTM(units=50, activation="tanh"))
    else:
        model.add(GRU(units=50, activation="tanh"))
    
    model.add(Dense(units=1))
    model.compile(optimizer="adam", loss="mse")
    
    early_stop = keras.callbacks.EarlyStopping(monitor='loss', patience=10)
    model.fit(X_tr, y_tr, epochs=100, batch_size=8, callbacks=[early_stop], verbose=0)
    
    pred = model.predict(X_te, verbose=0)
    pred = sc.inverse_transform(pred)
    actual_vals = testing_data[n_steps:]
    
    rmse = return_rmse(actual_vals, pred)
    return rmse

In [ ]:
sequence_lengths = [5, 10, 15, 20]
results = {"SimpleRNN": [], "LSTM": [], "GRU": []}

for n in sequence_lengths:
    print(f"Testing sequence length: {n}")
    for model_type in ["SimpleRNN", "LSTM", "GRU"]:
        rmse = train_and_evaluate(n, model_type)
        results[model_type].append(rmse)
        print(f"  {model_type}: RMSE = {rmse:.2f}")

In [ ]:
plt.figure(figsize=(10, 5))
for model_type in results:
    plt.plot(sequence_lengths, results[model_type], marker='o', label=model_type)
plt.xlabel("Sequence Length")
plt.ylabel("RMSE")
plt.title("RMSE vs Sequence Length")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
print("\nResults Summary:")
print("-" * 50)
print(f"{'Seq Length':<12}", end="")
for model_type in results:
    print(f"{model_type:<12}", end="")
print()
print("-" * 50)
for i, n in enumerate(sequence_lengths):
    print(f"{n:<12}", end="")
    for model_type in results:
        print(f"{results[model_type][i]:<12.2f}", end="")
    print()